In [4]:
import pandas as pd
import numpy as np

In [5]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [6]:
# Reading datasets
#movies.info()
#credits.info()

In [7]:
# Merge both data-set
movies = movies.merge(credits,on='title')

In [8]:
# Reading Merged dataset
#movies.head()
#movies.info()

In [9]:
# Removing unwanted data
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
#movies.head()

In [10]:
# checking the null values
movies.isnull().sum()
# droping null values
movies.dropna(inplace=True)

In [11]:
# checking duplicate rows
movies.duplicated().value_counts()
# droping duplicate values
movies.drop_duplicates(inplace=True)

In [12]:
# checking genres
movies.iloc[0].genres
# creating function to create useful genres
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        return L
# converting genres string list into normal list
# for that we use function ast.literal_eval()(

In [13]:
# Applying convert function in genres
movies['genres'] = movies['genres'].apply(convert)
# checking
#movies.head()

In [14]:
# Applying convert function in keywords
movies['keywords'] = movies['keywords'].apply(convert)

In [15]:
# Making usefull cast list - extracting top 3 cast
import ast

def convert_cast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L
# Applying function convert_cast in ['cast']
movies['cast'] = movies['cast'].apply(convert_cast)

In [16]:
# checking genres
movies.iloc[0].crew
# Making usefull crew list - fetching director only
import ast

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L
# Applying function convert_cast in ['cast']
movies['crew'] =movies['crew'].apply(fetch_director)

In [17]:
# converting dtype - string of overview into list of overiew
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [18]:
# checking the null values
movies['genres'].isnull().sum()
# droping null values
movies.dropna(inplace=True)

In [19]:
# Removing space between names to make single tag
# Removing space in genre,cast,crew
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [20]:
# Adding overview, genres, keywords, cast, crew to make Tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
# Making new dataframe
new_df = movies[['movie_id','title','tags']]

In [21]:
# Converting tag list into string
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\saksh\AppData\Local\Temp\ipykernel_16224\1190696506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [22]:
# converting tags in lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\saksh\AppData\Local\Temp\ipykernel_16224\287392959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [23]:
# Making words vector and removing stop words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vectors = cv.fit_transform(new_df['tags']).toarray()

# Checking vectors
#vectors

# looking all 5000 word by using .get_feature_names_out()
#cv.get_feature_names_out()

In [24]:

# Doing Stemming in tags by ntlk library using PorterStemmer class
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# function to stem every word present in text
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

# Appling stemming on ['tags']
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\saksh\AppData\Local\Temp\ipykernel_16224\893516160.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [25]:
# Using sklearn to find similar movies on basis of angle btw movies vector
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

# checking cosine similarity for 2nd movie
#similarity[1]

In [26]:

# Function give recommendation of similar movies
def recommendation(movies):
    # Getting movie name index
    movie_index = new_df[new_df['title']== str(movies)].index[0]
    # getting cosine similarities btw movies
    distances = similarity[movie_index]
    # sorting and getting top 5 similar movies
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [28]:
recommendation(input('Enter movie name :: '))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
import pickle
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [30]:
pickle.dump(similarity,open('similarity.pkl','wb'))